**LUIS IMLAUER | 2023 | limlauer.github.io**
# CS:GO Major players analysis
# Part 2

**WHO IS THE CS:GO GOAT?**
Majors are the biggest tournaments in CS:GO, and with the launch of CS2, we saw the last one forever. It's time to collect the data and analyze everything we can and answer some questions.

In [14]:
import pandas as pd
from selenium import webdriver
driver = webdriver.Chrome()
from bs4 import BeautifulSoup
pd.options.display.max_columns = 20
pd.options.display.max_rows = 2500


In [15]:
df = pd.read_csv('basic_players_info.csv') # This is the file we saved in the previous notebook
df.head()

,name,nationality,team,maps,rounds,KD-diff,KD,rating,event,event_ID,Full name,Player ID,Full link
0,ZywOo,France,Vitality,10,277,92,1.60,1.39,BLAST.tv Paris Major 2023,6973,Mathieu 'ZywOo' Herbaut,11893,https://www.hltv.org/player/11893/zywoo
1,iM,Romania,GamerLegion,12,306,68,1.34,1.35,BLAST.tv Paris Major 2023,6973,Mihai 'iM' Ivan,14759,https://www.hltv.org/player/14759/im
2,Spinx,Israel,Vitality,10,277,50,1.30,1.24,BLAST.tv Paris Major 2023,6973,Lotan 'Spinx' Giladi,18221,https://www.hltv.org/player/18221/spinx
3,NAF,Canada,Liquid,9,245,32,1.22,1.24,BLAST.tv Paris Major 2023,6973,Keith 'NAF' Markovic,8520,https://www.hltv.org/player/8520/naf
4,YEKINDAR,Latvia,Liquid,9,245,30,1.19,1.20,BLAST.tv Paris Major 2023,6973,Mareks 'YEKINDAR' GaÄ¼inskis,13915,https://www.hltv.org/player/13915/yekindar


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1521 entries, 0 to 1520
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   name         1521 non-null   object 
 1   nationality  1521 non-null   object 
 2   team         1521 non-null   object 
 3   maps         1521 non-null   int64  
 4   rounds       1521 non-null   int64  
 5   KD-diff      1521 non-null   int64  
 6   KD           1521 non-null   float64
 7   rating       1521 non-null   float64
 8   event        1521 non-null   object 
 9   event_ID     1521 non-null   int64  
 10  Full name    1521 non-null   object 
 11  Player ID    1521 non-null   int64  
 12  Full link    1521 non-null   object 
dtypes: float64(2), int64(5), object(6)
memory usage: 154.6+ KB


In [16]:
df_individuals = df[['name', 'Player ID', 'event_ID']]
df_individuals.head()

,name,Player ID,event_ID
0,ZywOo,11893,6973
1,iM,14759,6973
2,Spinx,18221,6973
3,NAF,8520,6973
4,YEKINDAR,13915,6973


In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1521 entries, 0 to 1520
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   name         1521 non-null   object 
 1   nationality  1521 non-null   object 
 2   team         1521 non-null   object 
 3   maps         1521 non-null   int64  
 4   rounds       1521 non-null   int64  
 5   KD-diff      1521 non-null   int64  
 6   KD           1521 non-null   float64
 7   rating       1521 non-null   float64
 8   event        1521 non-null   object 
 9   event_ID     1521 non-null   int64  
 10  Full name    1521 non-null   object 
 11  Player ID    1521 non-null   int64  
 12  Full link    1521 non-null   object 
dtypes: float64(2), int64(5), object(6)
memory usage: 154.6+ KB


Great, now let's add more information, specifically the individual stats from a player in a given event.
(The example data is what we pulled from dupreeh in the last major)

We will be adding:
1. OVERALL STATS
- Kills: 200
- Deaths: 233
- Kill / Death: 0.86
- Kill / Round: 0.61
- Rounds with kills: 138
- Kill - Death difference: K - D diff.

2. OPENING STATS
- Total opening kills: 31
- Total opening deaths: 44
- Opening kill ratio: 0.70
- Opening kill rating: 0.91
- Team win percent after first kill: 80.6%
- First kill in won rounds: 15.2%

3. ROUND STATS
- 0 kill rounds: 192
- 1 kill rounds: 95
- 2 kill rounds: 26
- 3 kill rounds: 15
- 4 kill rounds: 2
- 5 kill rounds: 0

4. WEAPON STATS
- Rifle kills: 153
- Sniper kills: 1
- SMG kills: 15
- Pistol kills: 31
- Grenade: 0
- Other: 1

This time there each row will have new observations, so it will take a lot more time! So let's make another function that let's us split the process in pieces like before and not overwrite any data

In [17]:
all_columns = ['Kills','Deaths','Kill / Death','Kill / Round','Rounds with kills','Kill - Death difference',
'Total opening kills','Total opening deaths','Opening kill ratio','Opening kill rating','Team win percent after first kill','First kill in won rounds',
'0 kill rounds','1 kill rounds','2 kill rounds','3 kill rounds','4 kill rounds','5 kill rounds',
'Rifle kills','Sniper kills','SMG kills','Pistol kills','Grenade','Other', 'Event link']

In [18]:
for column in all_columns:
    df_individuals[column] = None
df_individuals.head()

C:\Users\Luis\AppData\Local\Temp\ipykernel_69748\1065688594.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_individuals[column] = None
C:\Users\Luis\AppData\Local\Temp\ipykernel_69748\1065688594.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_individuals[column] = None
C:\Users\Luis\AppData\Local\Temp\ipykernel_69748\1065688594.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cavea

,name,Player ID,event_ID,Kills,Deaths,Kill / Death,Kill / Round,Rounds with kills,Kill - Death difference,Total opening kills,...,3 kill rounds,4 kill rounds,5 kill rounds,Rifle kills,Sniper kills,SMG kills,Pistol kills,Grenade,Other,Event link
0,ZywOo,11893,6973,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,iM,14759,6973,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,Spinx,18221,6973,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,NAF,8520,6973,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,YEKINDAR,13915,6973,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [27]:
df_individuals.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1521 entries, 0 to 1520
Data columns (total 28 columns):
 #   Column                             Non-Null Count  Dtype 
---  ------                             --------------  ----- 
 0   name                               1521 non-null   object
 1   Player ID                          1521 non-null   int64 
 2   event_ID                           1521 non-null   int64 
 3   Kills                              0 non-null      object
 4   Deaths                             0 non-null      object
 5   Kill / Death                       0 non-null      object
 6   Kill / Round                       0 non-null      object
 7   Rounds with kills                  0 non-null      object
 8   Kill - Death difference            0 non-null      object
 9   Total opening kills                0 non-null      object
 10  Total opening deaths               0 non-null      object
 11  Opening kill ratio                 0 non-null      object
 12  Openin

In [19]:
empty_individual_df = pd.DataFrame(columns = df_individuals.columns)
print(empty_individual_df)

Empty DataFrame
Columns: [name, Player ID, event_ID, Kills, Deaths, Kill / Death, Kill / Round, Rounds with kills, Kill - Death difference, Total opening kills, Total opening deaths, Opening kill ratio, Opening kill rating, Team win percent after first kill, First kill in won rounds, 0 kill rounds, 1 kill rounds, 2 kill rounds, 3 kill rounds, 4 kill rounds, 5 kill rounds, Rifle kills, Sniper kills, SMG kills, Pistol kills, Grenade, Other, Event link]
Index: []

[0 rows x 28 columns]


In [20]:
# This script below took me hours to code and test, and I should have used regex for everything (I think); but as soon as it worked I just wanted to keep going, I'll maybe fix it in the future
def extract_individual_stats(csv_name, Player_Names_List, Overwrite, Start_index = 0, Ending_index = 0, File_save_iter = 5):
    try:
        loaded_df = pd.read_csv(csv_name)
        print("Loaded existing CSV.")
    except FileNotFoundError:
        print("CSV file not found. Creating a new DataFrame.")
        loaded_df = empty_individual_df
    Output_DF = loaded_df
    
    if (Ending_index == 0):
        Ending_index = len(Player_Names_List)

    print(f"Preparing to search for {Ending_index} players...")
    processing_list = Player_Names_List[Start_index:Ending_index]

    existing_player_ids = list(Output_DF['Player ID']) # EXISTENTES
    existing_event_ids = list(Output_DF['event_ID']) # EXISTENTES
    total_rows = len(Player_Names_List) + 1
    save_index = 0 # File_save_iter is the max


    for index, row in processing_list.iterrows():
        player_id = row['Player ID']
        event_id = row['event_ID']
        player_name = row['name']

        if (not Overwrite) and (player_id in existing_player_ids) and (event_id in existing_event_ids):
            print(f"[{index}/{Ending_index} | {total_rows}] Skipping Player ID {player_id} in event {event_id} (already exists)")
            continue
        
        URL = f"https://www.hltv.org/stats/players/individual/{player_id}/{player_name.lower()}?event={event_id}"
        driver = webdriver.Chrome()
        driver.get(URL)
        page_source = driver.page_source
        driver.quit()

        soup1 = BeautifulSoup(page_source, "html.parser")
        soup2 = BeautifulSoup(soup1.prettify(), "html.parser")

        stats_rows = soup2.find_all('div', class_='stats-row')
        stat_dict = {}  # Create a new empty dictionary
        print(f"[{index}/{Ending_index} | {total_rows}] Extracting data for {player_name}...")
        stat_dict['name'] = player_name
        stat_dict['Player ID'] = player_id
        stat_dict['event_ID'] = event_id
        stat_dict['Event link'] = f"https://www.hltv.org/stats/players/individual/{player_id}/{player_name}?event={event_id}"
        for row in stats_rows:
            stat_name = row.find('span').text.strip()
            stat_value = row.find_all('span')[1].text.strip()
            stat_dict[stat_name] = stat_value  # Add to dictionary
        
        stat_dict['Kill - Death difference'] = int(stat_dict['Kills']) - int(stat_dict['Deaths'])

        Output_DF = pd.concat([Output_DF, pd.DataFrame([stat_dict])], ignore_index=True)
        print(f"[{index}/{Ending_index} | {total_rows}] Added statistics for Player {player_name} with ID {player_id} in event {event_id}")
        save_index += 1
        if save_index >= File_save_iter:
            Output_DF.to_csv("player_individual_stats.csv", index=False)
            print(f"[SAVED] > File saved after {File_save_iter} iterations, as player_individual_stats.csv")
            save_index = 0
        
    Output_DF.to_csv("player_individual_stats.csv", index=False)
    print("[SAVED] > File saved as player_individual_stats.csv")

In [21]:
extract_individual_stats("player_individual_stats.csv", df_individuals, False, Ending_index=1500, File_save_iter=5) # Overwrite function is not implemented yet

Loaded existing CSV.
Preparing to search for 1500 players...
[0/1500 | 1521] Skipping Player ID 11893 in event 6973 (already exists)
[1/1500 | 1521] Skipping Player ID 14759 in event 6973 (already exists)
[2/1500 | 1521] Skipping Player ID 18221 in event 6973 (already exists)
[3/1500 | 1521] Skipping Player ID 8520 in event 6973 (already exists)
[4/1500 | 1521] Skipping Player ID 13915 in event 6973 (already exists)
[5/1500 | 1521] Skipping Player ID 18053 in event 6973 (already exists)
[6/1500 | 1521] Skipping Player ID 10994 in event 6973 (already exists)
[7/1500 | 1521] Skipping Player ID 3741 in event 6973 (already exists)
[8/1500 | 1521] Skipping Player ID 19592 in event 6973 (already exists)
[9/1500 | 1521] Skipping Player ID 11816 in event 6973 (already exists)
[10/1500 | 1521] Skipping Player ID 7964 in event 6973 (already exists)
[11/1500 | 1521] Skipping Player ID 19763 in event 6973 (already exists)
[12/1500 | 1521] Skipping Player ID 9436 in event 6973 (already exists)
[13/

This process will take at least 5 hours, I'll run it when I go to sleep.

In [24]:
extract_individual_stats("player_individual_stats.csv", df_individuals, False, Ending_index=2000, File_save_iter=5) # Overwrite function is not implemented yet

Loaded existing CSV.
Preparing to search for 2000 players...
[0/2000 | 1521] Skipping Player ID 11893 in event 6973 (already exists)
[1/2000 | 1521] Skipping Player ID 14759 in event 6973 (already exists)
[2/2000 | 1521] Skipping Player ID 18221 in event 6973 (already exists)
[3/2000 | 1521] Skipping Player ID 8520 in event 6973 (already exists)
[4/2000 | 1521] Skipping Player ID 13915 in event 6973 (already exists)
[5/2000 | 1521] Skipping Player ID 18053 in event 6973 (already exists)
[6/2000 | 1521] Skipping Player ID 10994 in event 6973 (already exists)
[7/2000 | 1521] Skipping Player ID 3741 in event 6973 (already exists)
[8/2000 | 1521] Skipping Player ID 19592 in event 6973 (already exists)
[9/2000 | 1521] Skipping Player ID 11816 in event 6973 (already exists)
[10/2000 | 1521] Skipping Player ID 7964 in event 6973 (already exists)
[11/2000 | 1521] Skipping Player ID 19763 in event 6973 (already exists)
[12/2000 | 1521] Skipping Player ID 9436 in event 6973 (already exists)
[13/

### 4. Who has the most kills

### 5. Who has the least kills

### 6. Who has the most deaths

### 7. Who are the majors MVPs and what role did they have

### 8. What country has the players with most majors

### 9. How did the 'online era' affect players and teams

### 10. How did the Ukraine war affect players

### 11. Winners of every major

### 12. Who has the most majors

### 13. Who performed better accross all majors (In total and in average)

### 14. Who is the CS:GO GOAT?